# Advent of Code 2016

## Day 19: An Elephant Named Joseph 


https://adventofcode.com/2016/day/19


Each Elf brings a present. They all sit in a circle, numbered starting with position 1. Then, starting with the first Elf, they take turns stealing all the presents from the Elf to their left. An Elf with no presents is removed from the circle and does not take turns.

For example, with five Elves (numbered 1 to 5):

```
  1
5   2
 4 3
```

    Elf 1 takes Elf 2's present.
    Elf 2 has no presents and is skipped.
    Elf 3 takes Elf 4's present.
    Elf 4 has no presents and is also skipped.
    Elf 5 takes Elf 1's two presents.
    Neither Elf 1 nor Elf 2 have any presents, so both are skipped.
    Elf 3 takes Elf 5's three presents.

So, with five Elves, the Elf that sits starting in position 3 gets all the presents.

With the number of Elves given in your puzzle input, which Elf gets all the presents?

It seems trivial, but there is a sting in the tail. 

Let's try the naive solution first. Create a list of numbers representing the elves, and just loop around that, deleting any zeros until we have a single elf left

In [37]:
def steal_next(elfcount):
    circle = list(range(1, elfcount+1))

    target = 0
    while elfcount > 1:
        target = (target+1) % elfcount
        del circle[target]
        elfcount -= 1

    return circle[0]

We can test that on the example given:

In [38]:
steal_next(5)

3

Yay! Winner! Let's try on the actual data we're given, `3_014_603`:

In [3]:
steal_next(3_014_603)

KeyboardInterrupt: 

Hmm, that's not looking good.

We've fallen down a classic AoC trap door.

In Python, deleting an element from a list is an exceedingly expensive operation, as a large chunk of the list will have to be copied, element by element, to fill the void. For small lists, it's of no consequence. But in our case for every elf removed we have to shuffle millions of list entries.

A Python list is an inappropriate data structure for any algorithm making many deletions.

So this suddenly doesn't look quite so easy.

What data structures offer fast deletes? Trees, graphs, linked lists. We could solve the problem with a circular linked list for sure.

However, there is another way -- and that is to use the `blist` Python module.

In [19]:
import sys
!conda install --yes --prefix {sys.prefix} blist

Solving environment: done

# All requested packages already installed.



This `blist` module gives us a list-like data structure which is implemented as a tree under the hood. It's almost as fast as the native list for indexed access, but whilst also giving us tree-based speed for delete operations. A one-character tweak is required in `steal_next`:

In [8]:
from blist import blist

In [20]:
def steal_next(elfcount):
    circle = blist(range(1, elfcount+1))   # Make a blist instead of a list

    target = 0
    elfcount = len(circle)
    while elfcount > 1:
        target = (target+1) % elfcount
        del circle[target]
        elfcount -= 1

    return circle[0]

In [29]:
assert steal_next(3_014_603) == 1834903

Boom! Blist-ering speeds! Gold star! The linked-list implementation left as an exercise for the interested reader.



## Part 2

Very similar, only a different elf is chosen to steal from. Using the `blist` again, this runs fast

In [17]:
def steal_opposite(elfcount=3_014_603):
    circle = blist(range(1, elfcount+1))

    pos = 0
    elfcount = len(circle)
    while elfcount > 1:
        pos %= elfcount
        target = (pos + elfcount // 2) % elfcount
        del circle[target]
        if target > pos:
            pos += 1
        elfcount -= 1

    return circle[0]

In [28]:
assert steal_opposite(3_014_603) == 1420280

And that concludes day 19. Lesson of the day: if you're applying brute force and ignorance, understand the behaviour of your data structures.

A similar lesson is dealt out in https://adventofcode.com/2018/day/9

### Note

Using the `blist` approach works perfectly, but if we had decided to solve part 1 with a linked list, we'd be punished in part 2, as we can no longer find the item to remove without stepping through half the list, so a more sophisticated structure would be required.

# Analytical solution

This is the so-called `Josephus` problem: https://en.wikipedia.org/wiki/Josephus_problem, hence the title. There is an excellent deep-dive on the Numberphile YouTube channel on this problem: https://www.youtube.com/watch?v=uCsD3ZGzMgE

The Josephus probem has clever solutions. Part 1 is given in the Numberphile video above:

In [ ]:
def part1(n=3_014_603):
    return int(f"{n:b}"[1:] + "1", 2)

In [ ]:
assert part1() == 1834903

Part 2 is the general Josephus problem, which can be solved as a recurrence given in the Wikipedia article. Here's a Pythonification:

In [35]:
def f(n, p=1):
    """
    Analytical solution to the generic Josephus problem. 
    See Wikipedia page https://en.wikipedia.org/wiki/Josephus_problem
    """
    while 3*p <= n: 
        p *= 3
    if n == p: 
        return n
    return n - p + max(n - 2*p, 0)

In [36]:
f(3_014_603)

1420280